In [2]:
import pandas as pd
import numpy as np
from gensim.corpora.dictionary import Dictionary
import spacy as sp
import torch
from eda import lem
import pickle

In [3]:
vec_model= sp.load('en_core_web_lg')
df = pd.read_csv(r'./base_data.csv')
parse_list=lambda s:eval(s)
des = df.Description
cats = df.Categories.apply(parse_list)

FileNotFoundError: [Errno 2] No such file or directory: './filtered_apis.csv'

In [6]:
from nltk import pos_tag_sents
pos_tags = pos_tag_sents(des.apply(lem))

In [8]:
# nn_sents = pd.Series([' '.join([e[0] for e in li]) for li in pos_tags])
# nn_sents.dropna()

0        rebro kommun rebro municipality sweden provide...
1        csfd api program interface interact czech slov...
2        ternity decentralize blockchain system applica...
3        api provide method source liquidity mesh kyber...
4        fact russian website dedicate help people beco...
                               ...                        
20758    zync upwire api enable developer set event tri...
20759    zyncro enterprise social networking service pr...
20760    zynx health api provide access zynx content fa...
20761    zype api integrate video feature application a...
20762    zypr application api design work across variet...
Length: 20763, dtype: object

In [7]:
nn_words = []
def dup_nn(li):
    res = ''
    nns = set()
    for w,att in li:
        if att.startswith('NN') or att.startswith('JJ'):
            res += w+' '
            nns.add(w)
        # res += w+' '
    nn_words.append(nns)
    return res
nn_sents = pd.Series([dup_nn(sent) for sent in pos_tags])

In [8]:
# nn_docs = nn_sents.apply(lambda x:vec_model(x))
nn_docs = cats.apply(lambda x:' '.join(set(x))).apply(vec_model)
def get_most_sim(sent_doc,nns,topn = 3):
    words = []
    if not sent_doc:
        return []
    if len(sent_doc)<2:
        return []
    # words = [ (w,max([tag.similarity(vec_model.vocab[w]) for tag in sent_doc])) for w in nns]
    for w in nns:
        ve = vec_model.vocab[w]
        sim = sent_doc.similarity(ve)
        words.append((w,sim))
    # words.sort(key= lambda x:x[1])
    return words
nn_en = pd.Series([[(t,)*round(5*wei) for t,wei in get_most_sim(doc,nns)] for doc,nns in zip(nn_docs,nn_words)])

<ipython-input-8-94de49c9adb3>:12: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  sim = sent_doc.similarity(ve)


1-将内容中名词简单重复
2-将所有名词按和tag的相似度来进行重复n次


In [9]:
nn_en = nn_en.apply(lambda x: ' '.join([' '.join(en) for en in x]))
nn_sents = nn_sents+' '+nn_en
df['des'] = nn_sents


In [10]:
nn_sents[0]

'rebro kommun rebro municipality sweden provide portal government data information waste recycle library transit politics sport culture business tourism much information map restful api data xml json site documentation swedish  documentation documentation culture culture site sport sport government government government sweden sweden api much much map map   waste waste swedish xml  provide provide recycle data data data data transit transit portal portal library library municipality municipality politics politics tourism tourism business business information information information '

In [11]:
nn_vecs = nn_sents.apply(vec_model)

In [12]:
rel_cats = [[(t,d.similarity(vec_model.vocab[t])) for t in tags] for d,tags in zip(nn_vecs,cats)]

<ipython-input-12-3b118e43b4a0>:1: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  rel_cats = [[(t,d.similarity(vec_model.vocab[t])) for t in tags] for d,tags in zip(nn_vecs,cats)]


In [13]:
df['new_cats']= [set(x[0] for x in list(filter(lambda x:x[1]>0.5,li))) for li in rel_cats]

In [14]:
df.to_csv('processed.csv',index=None)

In [15]:
dddd = pd.read_csv('processed.csv')

In [68]:
dddd

,Name,Description,Categories,new_cats,des
0,Ã–rebro kommun,rebro kommun rebro municipality Sweden ...,"['government', 'data', 'data', 'nordic']","{'government', 'data'}",rebro kommun rebro municipality sweden provide...
1,ÄŒSFD,The CSFD API is a programming interface to int...,"['media', 'search', 'database']","{'search', 'database', 'media'}",csfd api program interface interact czech slov...
2,Æternity REST API v2,ternity is a decentralized blockchain system ...,"['blockchain', 'bitcoin', 'community']",{'community'},ternity blockchain system application educatio...
3,0x,The 0x API provides methods to source liquidit...,"['blockchain', 'cryptocurrency', 'currency', '...","{'currency', 'financial'}",api provide method source liquidity mesh kyber...
4,100 Facts About Me,100 Facts About Me is a Russian website dedica...,"['social', 'russian']",{'social'},fact russian website dedicate help people good...
...,...,...,...,...,...
20758,Zync Upwire,The Zync Upwire API enables developers to set ...,"['messaging', 'campaigns', 'email', 'voice']",{'messaging'},zync upwire api enable developer event trigger...
20759,Zyncro,Zyncro is an enterprise social networking serv...,"['enterprise', 'social', 'file sharing', 'blog...","{'social', 'enterprise'}",zyncro enterprise social networking service fe...
20760,Zynx Health,The Zynx Health API provides access to Zynx Co...,"['healthcare', 'content', 'data', 'developers'...","{'healthcare', 'financial', 'management', 'dat...",zynx health api access zynx content healthcare...
20761,Zype,The Zype API integrates video features into ap...,"['video', 'analytics', 'business', 'platform-a...",{'business'},zype api integrate video feature application a...
